In [2]:
# -*- coding: utf-8 -*-
"""
Created on Wed Jun  5 13:37:54 2019

@author: aw1001
"""
import time
from datetime import datetime
import cv2
import base64
import pandas as pd
import json
import requests
import datetime
import numpy as np
import datetime
import re
import dask.dataframe as dd
import sqlite3 as sqlite
import time

#conexão com o webservice
#url_cebrace_salvar = "http://sistemas.cebrace.com.br/EzTask.WebAPI/api/ocorrencia/salvar"
#url_cebrace_update = "http://sistemas.cebrace.com.br/EzTask.WebAPI/api/ocorrencia/id/adicionar-foto"

url_cebrace_salvar = "http://localhost:22700/api/ocorrencia/salvar"
url_cebrace_update = "http://localhost:22700/api/ocorrencia/id/adicionar-foto"
url_cebrace_connect = 'http://localhost:22700/api/token'
#headers_cebrace = {"content-type": "application/x-www-form-urlencoded"}

In [127]:
import bd_connect

DatabaseError: 2003 (HY000): Can't connect to MySQL server on 'localhost' (10061)

In [3]:
conn = sqlite.connect('safety_camera1.db')
cursor = conn.cursor()
df = pd.read_sql_query("SELECT * FROM images Where status = 10", conn, index_col =None)

In [65]:
def connect_webservice_cebrace(username, password):
    #url_cebrace_connect = "http://sistemas.cebrace.com.br/EzTask.WebAPI/api/token"
    headers_cebrace_connect = { "content-type": "application/x-www-form-urlencoded" }
    data_cebrace_connect = { "grant_type": "password", "username": username, "password": password }
    response_conect = requests.post(url_cebrace_connect, data = data_cebrace_connect, headers = headers_cebrace_connect)
    response_conect_json = response_conect.json()

    return response_conect_json["access_token"], response_conect_json["expires_in"]

access_token, expires_in = connect_webservice_cebrace("KEYRUSAPI", "SW2019")
headers_cebrace = { "content-type": "application/json", "Authorization": ("Bearer " + access_token) }

In [66]:
#dataframe =  'bancodeImagens1.csv'
#df =  pd.read_csv(dataframe,
                  #sep = ',', 
                  #error_bad_lines = False,
                #encoding='utf-8')

In [67]:
#df = df.reset_index(drop=True)
#df.head()
df

,idimage,idcamera,acquisition_time,update_time,status,json_returned,eztask_return,raw_image,processed_image,anomaly_group,accuracy_check,acquisition_time_group
0,4,4,2019-06-25 08:02:00,2019-06-28 10:53:52,10,"{""item_0"": {""top"": 387, ""left"": 1184, ""class"":...","{""id"": 2054, ""mensagem"": ""Ocorrência cadastrad...",/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...,bytearray(b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,None,0.0,0
1,11,4,2019-06-25 08:02:00,2019-06-28 10:53:52,10,"{""item_0"": {""top"": 387, ""left"": 1184, ""class"":...","{""id"": 2055, ""mensagem"": ""Ocorrência cadastrad...",/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...,bytearray(b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,None,0.0,0
2,18,4,2019-06-25 08:02:00,2019-06-28 10:53:53,10,"{""item_0"": {""top"": 389, ""left"": 1187, ""class"":...","{""id"": 2056, ""mensagem"": ""Ocorrência cadastrad...",/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...,bytearray(b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,None,0.0,0
3,25,4,2019-06-25 08:02:00,2019-06-28 10:53:53,10,"{""item_0"": {""top"": 389, ""left"": 1190, ""class"":...","{""id"": 2057, ""mensagem"": ""Ocorrência cadastrad...",/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...,bytearray(b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,None,0.0,0
4,32,4,2019-06-25 08:02:00,2019-06-28 10:53:54,10,"{""item_0"": {""top"": 389, ""left"": 1190, ""class"":...","{""id"": 2058, ""mensagem"": ""Ocorrência cadastrad...",/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...,bytearray(b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,None,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
115,2024,1,2019-06-28 09:22:00,2019-06-28 10:54:18,10,"{""item_0"": {""top"": 724, ""left"": 1844, ""class"":...","{""id"": 2111, ""mensagem"": ""Ocorrência cadastrad...",/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...,bytearray(b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,None,0.0,10
116,2028,5,2019-06-28 09:22:00,2019-06-28 10:54:18,10,"{""item_0"": {""top"": 672, ""left"": 1675, ""class"":...","{""id"": 2112, ""mensagem"": ""Ocorrência cadastrad...",/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...,bytearray(b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,None,0.0,10
117,2031,1,2019-06-28 09:22:00,2019-06-28 10:54:19,10,"{""item_0"": {""top"": 724, ""left"": 1844, ""class"":...","{""id"": 2113, ""mensagem"": ""Ocorrência cadastrad...",/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...,bytearray(b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,None,0.0,10
118,2035,5,2019-06-28 09:22:00,2019-06-28 10:54:19,10,"{""item_0"": {""top"": 672, ""left"": 1677, ""class"":...","{""id"": 2114, ""mensagem"": ""Ocorrência cadastrad...",/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...,bytearray(b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,None,0.0,10


In [72]:
now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
#print('acquisition_time XXXXXXX', df['acquisition_time'].iloc[:1][0])
df['acquisition_time'] = pd.to_datetime(df.acquisition_time)
df['acquisition_time'] = df['acquisition_time'].dt.floor('120s')
df['acquisition_time_group'] = df['acquisition_time'].factorize()[0]
#agrupamentosCamera = df.groupby('acquisition_time_group', axis= 0 )
agrupamentosCamera = df.groupby(['acquisition_time_group'], axis= 0 )

In [75]:
'''
Factorize, retorna o valor distinto dentro do conjunto de dados.
'''
valorRequisicao = 0
valorEnvioFoto = 0
# itera em cada grupo
for indiceAgrupamento, agrupamentoCamera in agrupamentosCamera:  
    #print(agrupamentoCamera.iterrows())
    valorRequisicao += 1
    primeiroRegistroCamera = agrupamentoCamera
    #print(agrupamentoCamera.iterrows())
    #print(primeiroRegistroCamera)
    idcamera = str(primeiroRegistroCamera['idcamera'].values[0])
    #print((idcamera))
    #acquisition_time = str(primeiroRegistroCamera['acquisition_time'])
    
    acquisition_time_formatted = pd.to_datetime(str(primeiroRegistroCamera['acquisition_time'].values[0])).strftime('%Y-%m-%d %H:%M:%S')
    
    update_time = primeiroRegistroCamera['update_time']
    status = primeiroRegistroCamera['status']
    json_returned = primeiroRegistroCamera['json_returned']
    anomalies = json_returned
    #print(anomalies)
    anomaly_txt= ""
    
    for item in anomalies:
        if  re.findall(r'\b(\w*safety\w*)\b', item ) ==['safety']:
            anomaly_txt += " \nAndando fora da faixa de seguranca "
        elif re.findall(r'\b(\w*no helmet\w*)\b', item ) == ['no helmet']:
            anomaly_txt += " \nFalta de capacete "
        elif re.findall(r'\b(\w*no vest\w*)\b', item ) == ['no vest']:
            anomaly_txt += " \nFalta de colete "
    
    eztask_return = primeiroRegistroCamera['eztask_return']
    raw_image = primeiroRegistroCamera['raw_image']
    processed_image = primeiroRegistroCamera['processed_image']
    anomaly_group = primeiroRegistroCamera['anomaly_group']
    accuracy_check = primeiroRegistroCamera['accuracy_check']
    
    jpg_as_text = str(primeiroRegistroCamera['processed_image'])
    jpg_as_text = jpg_as_text.encode("UTF-8")
    jpg_as_text = base64.b64encode(jpg_as_text).decode('utf-8')
    
    data_cebrace =  {
        "dataInicial": acquisition_time_formatted,
        "dataFinal":now, 
        "camera": ("Unidade JCR - Camera: " + str(idcamera) ),
        "anomalia": anomaly_txt,
        "anexo": jpg_as_text
    }
    
    
    data_cebrace = json.dumps(data_cebrace)
    #print(data_cebrace)
    response_cebrace_salvar = requests.post(url_cebrace_salvar, data = data_cebrace, headers = headers_cebrace)
    print(response_cebrace_salvar.text)
    
    if response_cebrace_salvar.status_code != 200:
        
        break        
            
    else:
        ocorrenciaSalvaEzTask = json.loads(response_cebrace_salvar.text)
        #print(ocorrenciaSalvaEzTask['id'])
        
        imagensRestantesCamera = agrupamentoCamera[1:]
        #print(imagensRestantesCamera)
        
            
        for indice , imagemRestante in imagensRestantesCamera.iterrows(): 
            valorEnvioFoto += 1
            dadosEnvioImagemRestate =  json.dumps({"atividadeID": ocorrenciaSalvaEzTask['id'],"anexo" : jpg_as_text})
            idocorrencia =  str(ocorrenciaSalvaEzTask['id'])
            #print(idocorrencia)
            
            
            if imagemRestante.empty:
                break
            else:
                #jpg_as_text = imagemRestante['processed_image ']
                #jpg_as_text = jpg_as_text.encode("UTF-8")
                #jpg_as_text = base64.b64encode(jpg_as_text).decode('utf-8')
                #print(jpg_as_text)
                url_cebrace_update = url_cebrace_update.replace('id',idocorrencia)
                #print(url_cebrace_update)
                dadosEnvioImagemRestate =  json.dumps({
                                                      "anexo" : jpg_as_text,
                                                      #"anomalia": anomaly_txt
                                                      })
                print(dadosEnvioImagemRestate)
                response_cebrace_update = requests.put(url_cebrace_update,data = dadosEnvioImagemRestate,headers = headers_cebrace)

                respostaDadosEnvioImagemRestate =  json.loads(response_cebrace_update.text)
                #print("Resposta do sistema  :",respostaDadosEnvioImagemRestate['mensagem'])
            

    


{
  "id": 7261,
  "mensagem": "Ocorrência cadastrada com sucesso. \n Atividade: 403120. \n ID Ocorrencia: 7261"
}
{"anexo": "MCAgICAgYnl0ZWFycmF5KGInXHhmZlx4ZDhceGZmXHhlMFx4MDBceDEwSkZJRlx4MDBceC4uLgoxICAgICBieXRlYXJyYXkoYidceGZmXHhkOFx4ZmZceGUwXHgwMFx4MTBKRklGXHgwMFx4Li4uCjIgICAgIGJ5dGVhcnJheShiJ1x4ZmZceGQ4XHhmZlx4ZTBceDAwXHgxMEpGSUZceDAwXHguLi4KMyAgICAgYnl0ZWFycmF5KGInXHhmZlx4ZDhceGZmXHhlMFx4MDBceDEwSkZJRlx4MDBceC4uLgo0ICAgICBieXRlYXJyYXkoYidceGZmXHhkOFx4ZmZceGUwXHgwMFx4MTBKRklGXHgwMFx4Li4uCjUgICAgIGJ5dGVhcnJheShiJ1x4ZmZceGQ4XHhmZlx4ZTBceDAwXHgxMEpGSUZceDAwXHguLi4KNiAgICAgYnl0ZWFycmF5KGInXHhmZlx4ZDhceGZmXHhlMFx4MDBceDEwSkZJRlx4MDBceC4uLgo3ICAgICBieXRlYXJyYXkoYidceGZmXHhkOFx4ZmZceGUwXHgwMFx4MTBKRklGXHgwMFx4Li4uCjggICAgIGJ5dGVhcnJheShiJ1x4ZmZceGQ4XHhmZlx4ZTBceDAwXHgxMEpGSUZceDAwXHguLi4KOSAgICAgYnl0ZWFycmF5KGInXHhmZlx4ZDhceGZmXHhlMFx4MDBceDEwSkZJRlx4MDBceC4uLgoxMCAgICBieXRlYXJyYXkoYidceGZmXHhkOFx4ZmZceGUwXHgwMFx4MTBKRklGXHgwMFx4Li4uCjExICAgIGJ5dGVhcnJheShiJ1x4ZmZceGQ4XHhmZlx4ZTBceDAwXHg

KeyboardInterrupt: 